### Treehouse analysis stories coded up against the GA4GH RNASeq API

References:

https://ga4gh-schemas.readthedocs.io/en/latest/schemas/index.html

https://github.com/BD2KGenomics/bioapi-examples

https://github.com/ga4gh/ga4gh-client/blob/master/ga4gh/client/client.py

In [1]:
import pandas as pd
import numpy as np

!pip2 install -q --upgrade ga4gh --pre
import ga4gh
from ga4gh.client import client
print "ga4gh.client version", ga4gh.client.__version__

ga4gh.client version 0.0.4.dev9+ngd79dd00.d20170106


In [2]:
# See https://github.com/ga4gh/ga4gh-client/blob/master/ga4gh/client/client.py
c = client.HttpClient("http://1kgenomes.ga4gh.org")

# Experimental Recompute Server
# c = client.HttpClient("http://13.93.238.3/ga4gh/")

In [3]:
# Use the first dataset
dataset = c.search_datasets().next()
print "dataset:", dataset

dataset: id: "WyIxa2dlbm9tZXMiXQ"
name: "1kgenomes"
description: "Variants from the 1000 Genomes project and GENCODE genes annotations"



In [8]:
quantification_set = c.search_rna_quantification_sets(dataset_id=dataset.id).next()
print "quantification_set:", quantification_set

quantification_set: id: "WyIxa2dlbm9tZXMiLCJFLUdFVVYtMSBSTkEgUXVhbnRpZmljYXRpb24iXQ"
dataset_id: "WyIxa2dlbm9tZXMiXQ"
name: "E-GEUV-1 RNA Quantification"



Additional user stories -- things that right now we're implementing in pandas...

 - I have a mapping {feature -> threshold value} and a sample. I'd like to get a vector of (feature, expression value) for all features in that sample where expression value > feature[threshold]. 

- I'd like to get the features in a sample sorted by expression value.


- I have a set of sample identifiers. For each feature in samples in that set, I'd like to get statistics on its expression values within that set (median, quartiles, interquartile range).

Cheers,
Ellen

As a user I’d like to store 10k expression profiles plus phenotype information easily

As a user I’d like to extract all or a subset of those expression values into a large matrix easily by phenotype and/or gene/expression level

As a user I’d like to extract a single expression profile for Not1 analysis against the prior 2 requirements

#### Query by phenotype

"I'd like to generate a set of samples based on phenotype information (not just "disease = ALAL" but more complex queries, able to include or exclude samples by identifier)"

"Get the expression levels for all female Thyroid tumor samples"

In [13]:
%%time
# All males from the GBR population
biosample_ids = [b.id for b in c.search_biosamples(dataset_id=dataset.id) 
                 if b.info["Population"].values 
                 and "GBR" in [v.string_value for v in b.info["Population"].values]
                 and "male" in c.get_individual(b.individual_id).sex.term]
print "Samples found:", len(biosample_ids)

Samples found: 107
CPU times: user 3.34 s, sys: 24 ms, total: 3.37 s
Wall time: 9.52 s


In [98]:
rna_quantifications = [r for r in c.search_rna_quantifications(rna_quantification_set_id=quantification_set.id)
                       if r.biosample_id in biosample_ids]

In [112]:
expression = [{r.biosample_id: list(c.search_expression_levels(rna_quantification_id=r.id, 
                                              feature_ids= ["ENST00000619216.1", "ENST00000469289.1"]))
              for r in rna_quantifications}]

In [114]:
expression[0]

{u'WyIxa2dlbm9tZXMiLCJiIiwiSEcwMDA5NiJd': [id: "WyIxa2dlbm9tZXMiLCJFLUdFVVYtMSBSTkEgUXVhbnRpZmljYXRpb24iLCJIRzAwMDk2IiwiRU5TVDAwMDAwNjE5MjE2LjFIRzAwMDk2Il0"
  name: "ENST00000619216.1"
  feature_id: "ENST00000619216.1"
  rna_quantification_id: "WyIxa2dlbm9tZXMiLCJFLUdFVVYtMSBSTkEgUXVhbnRpZmljYXRpb24iLCJIRzAwMDk2Il0"
  raw_read_count: 1.0
  expression: 3.34066
  is_normalized: true
  units: TPM],
 u'WyIxa2dlbm9tZXMiLCJiIiwiSEcwMDA5OSJd': [id: "WyIxa2dlbm9tZXMiLCJFLUdFVVYtMSBSTkEgUXVhbnRpZmljYXRpb24iLCJIRzAwMDk5IiwiRU5TVDAwMDAwNjE5MjE2LjFIRzAwMDk5Il0"
  name: "ENST00000619216.1"
  feature_id: "ENST00000619216.1"
  rna_quantification_id: "WyIxa2dlbm9tZXMiLCJFLUdFVVYtMSBSTkEgUXVhbnRpZmljYXRpb24iLCJIRzAwMDk5Il0"
  raw_read_count: 1.0
  expression: 1.75
  is_normalized: true
  units: TPM],
 u'WyIxa2dlbm9tZXMiLCJiIiwiSEcwMDEwMCJd': [id: "WyIxa2dlbm9tZXMiLCJFLUdFVVYtMSBSTkEgUXVhbnRpZmljYXRpb24iLCJIRzAwMTAwIiwiRU5TVDAwMDAwNjE5MjE2LjFIRzAwMTAwIl0"
  name: "ENST00000619216.1"
  feature_id: "ENST0

In [4]:
# e = pd.DataFrame.from_records([{'feature_id': f.feature_id, 'expression': f.expression} 
#                                for f in feature], index='feature_id')
# e.head()
a = expression[0]["WyIxa2dlbm9tZXMiLCJiIiwiSEcwMDA5NiJd"][0]

NameError: name 'expression' is not defined